In [1]:
!pip install Optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.9 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import optuna
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
SEED=1234

In [60]:
# Cargar el conjunto de datos
datos = pd.read_csv("Temp_Asu20092021.csv")

In [61]:
# Convertir la columna 'Fecha' al tipo datetime y establecerla como índice
datos['Fecha'] = pd.to_datetime(datos['Fecha'])
datos.set_index('Fecha', inplace=True)

#PRIMER APORTE EN EL CODIGO
# Interpolar valores faltantes y reindexar para tener frecuencia horaria
datos = datos.resample('H').interpolate()

In [62]:
# Interpolar valores faltantes y reindexar para tener frecuencia horaria
datos = datos.resample('H').interpolate()

In [63]:
# Filtrar datos a partir de 2019
datos = datos[datos.index.year >= 2019]

In [64]:
# Calcular los percentiles 95, 50 (mediana) y 5 para cada día
percentiles_diarios = datos.resample('D')['Temperatura'].agg([lambda x: np.percentile(x, 95),
                                                              lambda x: np.percentile(x, 50),
                                                              lambda x: np.percentile(x, 5)])
percentiles_diarios.columns = ['Percentil_95', 'Mediana', 'Percentil_5']

In [65]:
# Desplazar para obtener los percentiles del siguiente día como variables objetivo
percentiles_diarios['Percentil_95_siguiente_dia'] = percentiles_diarios['Percentil_95'].shift(-1)
percentiles_diarios['Mediana_siguiente_dia'] = percentiles_diarios['Mediana'].shift(-1)
percentiles_diarios['Percentil_5_siguiente_dia'] = percentiles_diarios['Percentil_5'].shift(-1)

In [66]:
# Crear otras variables
#diferencias entre temperaturas horarias
datos['Temp_diff'] = datos['Temperatura'].diff()
#media móvil
datos['Temp_rolling_mean'] = datos['Temperatura'].rolling(window=24).mean()
#desviación estándar móvil
datos['Temp_rolling_std'] = datos['Temperatura'].rolling(window=24).std()

In [67]:
# Eliminar filas con valores faltantes
datos.dropna(inplace=True)
percentiles_diarios.dropna(inplace=True)


In [80]:
# Características originales: percentiles 95, mediana y 5
X_original = percentiles_diarios[['Percentil_95', 'Mediana', 'Percentil_5']]

# Características adicionales: diferencia de temperatura, media móvil y desviación estándar móvil
#X_nuevas = datos[['Temp_diff', 'Temp_rolling_mean', 'Temp_rolling_std']].resample('D').mean()

# Seleccionar características
X = pd.concat([X_original, X_nuevas], axis=1)
X = X.iloc[:-1]
#X= percentiles_diarios[['Percentil_95', 'Mediana', 'Percentil_5']]
y = percentiles_diarios[['Percentil_95_siguiente_dia','Mediana_siguiente_dia','Percentil_5_siguiente_dia']].values

In [81]:
# Escalar las características
scaler_X = StandardScaler()
X_escalado = scaler_X.fit_transform(X)

In [82]:
# Dividir datos en conjuntos de entrenamiento y validación
X_entrenamiento, X_validacion, y_entrenamiento, y_validacion = train_test_split(X_escalado, y, test_size=0.2, random_state=SEED, shuffle=False)

In [14]:
# Definir función objetivo para Optuna
def objetivo(trial):
    num_capas = 1
    tf.keras.utils.set_random_seed(SEED)

    num_capas = 1
    num_unidades = trial.suggest_categorical('num_unidades', [16, 32, 64])


    # Definir la arquitectura del modelo
    modelo = Sequential()
    modelo.add(Dense(num_unidades, activation='relu', input_shape=(X_entrenamiento.shape[1],)))
    for _ in range(num_capas - 1):
        modelo.add(Dense(num_unidades, activation='relu'))

    # Capa de salida para predecir los 3 percentiles (95, 50, 5)
    modelo.add(Dense(3, activation='linear'))

    # Compilar el modelo
    modelo.compile(optimizer=Adam(),
                   loss='mse',  # Para regresión, se usa 'mean squared error'
                   metrics=['mae'])  # Mean Absolute Error (MAE) para evaluación

    # Entrenar el modelo
    modelo.fit(X_entrenamiento, y_entrenamiento, validation_data=(X_validacion, y_validacion), epochs=30, batch_size=256, verbose=0, shuffle=False)

    # Evaluar el modelo en el conjunto de validación
    _, val_mae = modelo.evaluate(X_validacion, y_validacion, verbose=0)

    return val_mae

In [15]:
# Realizar la optimización de hiperparámetros usando Optuna
estudio = optuna.create_study(direction='minimize')  # Minimizamos el MAE en lugar de maximizar
estudio.optimize(objetivo, n_trials=10)

[I 2024-09-11 21:11:28,612] A new study created in memory with name: no-name-871f2db6-f592-4ee1-9b3e-e4f0ec1f3e2d
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
[I 2024-09-11 21:11:32,250] Trial 0 finished with value: 20.432693481445312 and parameters: {'num_unidades': 64}. Best is trial 0 with value: 20.432693481445312.
[I 2024-09-11 21:11:35,617] Trial 1 finished with value: 21.130168914794922 and parameters: {'num_unidades': 32}. Best is trial 0 with value: 20.432693481445312.
[I 2024-09-11 21:11:38,634] Trial 2 finished with value: 21.94194221496582 and parameters: {'num_unidades': 16}. Best is trial 0 with value: 20.432693481445312.
[I 2024-09-11 21:11:42,951] Trial 3 finished with value: 21.1301689147

In [18]:
# Obtener los mejores hiperparámetros
mejor_num_capas =  1
mejor_num_unidades = estudio.best_params['num_unidades']

print("Mejores Hiperparámetros:")
print("Número de Capas:", 1)
print("Número de Unidades:", mejor_num_unidades)

Mejores Hiperparámetros:
Número de Capas: 1
Número de Unidades: 64


In [19]:
# Entrenar el modelo final usando los mejores hiperparámetros
modelo_final = Sequential()
modelo_final.add(Dense(mejor_num_unidades, activation='relu', input_shape=(X_entrenamiento.shape[1],)))
for _ in range(mejor_num_capas - 1):
    modelo_final.add(Dense(mejor_num_unidades, activation='relu'))
modelo_final.add(Dense(3, activation='linear'))

modelo_final.compile(optimizer=tf.keras.optimizers.Adam(),
                     loss='mse',
                     metrics=['mae'])

In [20]:
# Entrenar el modelo final
modelo_final.fit(X_entrenamiento, y_entrenamiento, validation_data=(X_validacion, y_validacion), epochs=50, batch_size=256, verbose=1, shuffle=False)

Epoch 1/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - loss: 629.8170 - mae: 23.9941 - val_loss: 589.0117 - val_mae: 22.9169
Epoch 2/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 626.9188 - mae: 23.9338 - val_loss: 586.2498 - val_mae: 22.8544
Epoch 3/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 624.0053 - mae: 23.8731 - val_loss: 583.4566 - val_mae: 22.7911
Epoch 4/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 621.0536 - mae: 23.8115 - val_loss: 580.6237 - val_mae: 22.7267
Epoch 5/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 618.0551 - mae: 23.7488 - val_loss: 577.7426 - val_mae: 22.6611
Epoch 6/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 615.0005 - mae: 23.6848 - val_loss: 574.8023 - val_mae: 22.5939
Epoch 7/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 611.8785 - mae: 23.6192 - val_loss: 571.7953 - val_mae: 22.5250
Epoch 8/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 608.6756 - mae: 23.5516 - val_loss: 568.7111 - val_mae: 22.4539
Epoch 9/50
4/4 ━━━━━━━━━━━━━━━━━

In [21]:
# Evaluar en conjunto de prueba
mae = modelo_final.evaluate(X_validacion, y_validacion, verbose=0)
print(f'MAE en validación: {mae}')

MAE en validación: [336.75152587890625, 16.458694458007812]
